In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yfinance as yf
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    Float,
    DateTime,
    Integer
)
import sqlite3

## Create a helper function to insert data
`yfinance` has evolved to be able to download multiple tickers at a time but it becomes a nested multi-index dataframe. I prefer to download data for different tickers individually.

In [17]:
def insert_data(
    db_path: str,
    ticker: str,
    date_start: str,
    date_end: str,
) -> None:
    """Function to insert data into a SQLite  database."""
    
    try:
        data = yf.download(ticker, start=date_start, end=date_end, group_by="ticker")
    except Exception as e:
        raise e
    data = data.xs(ticker, axis=1, level="Ticker")
    data = data.reset_index()
    print(ticker + ' has '+ str(len(data))+ ' records')
    engine = create_engine(f"sqlite:///{db_path}")
    metadata_obj = MetaData()
    table = Table(
        ticker.lower(),
        metadata_obj,
        Column("id", Integer, primary_key=True),
        Column("date", DateTime),
        Column("open", Float),
        Column("high", Float),
        Column("low", Float),
        Column("close", Float),
        Column("volume", Integer),
    )
    metadata_obj.create_all(engine)
    conn = sqlite3.connect(db_path)
    data.to_sql(
        ticker.lower(),
        conn,
        if_exists='append',
        index=False
    )
    
    print(f"Ticker {ticker} added to database")
    
    cursor = conn.cursor()
    cursor.execute('SELECT count(*) '+ ticker.lower())
    rows = cursor.fetchall()
    print(f"Ticker {ticker} has {rows[0][0]} rows")

In [20]:
from datetime import datetime
from tqdm.notebook import tqdm

today = datetime.today().strftime("%Y-%m-%d")

In [21]:
import os

if not os.path.exists("../database"):
    os.makedirs("../database")

In [22]:
for ticker in tqdm(["ILMN", "AAPL", "NVDA"]):
    insert_data(
        db_path = "../database/stocks.db",
        ticker=ticker,
        date_start="2020-01-01",
        date_end=today
    )

  0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\320086703\AppData\Local\Temp\ipykernel_10836\1805404900.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=date_start, end=date_end, group_by="ticker")
[*********************100%***********************]  1 of 1 completed
C:\Users\320086703\AppData\Local\Temp\ipykernel_10836\1805404900.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=date_start, end=date_end, group_by="ticker")


ILMN has 1486 records
Ticker ILMN added to database
Ticker ILMN has 1 rows


[*********************100%***********************]  1 of 1 completed
C:\Users\320086703\AppData\Local\Temp\ipykernel_10836\1805404900.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=date_start, end=date_end, group_by="ticker")


AAPL has 1486 records
Ticker AAPL added to database
Ticker AAPL has 1 rows


[*********************100%***********************]  1 of 1 completed

NVDA has 1486 records
Ticker NVDA added to database
Ticker NVDA has 1 rows


In [29]:
ticker = 'AAPL'
date_start="2000-01-01"
date_end=datetime.today().strftime("%Y-%m-%d")
data = yf.download(ticker, start=date_start, end=date_end, group_by="ticker")
data = data.xs(ticker, axis=1, level="Ticker")
data = data.reset_index()
len(data)

C:\Users\320086703\AppData\Local\Temp\ipykernel_10836\3282134336.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=date_start, end=date_end, group_by="ticker")
[*********************100%***********************]  1 of 1 completed


6517

In [30]:
conn = sqlite3.connect("../database/stocks.db")
cursor = conn.cursor()
cursor.execute('SELECT count(*) aapl')
rows = cursor.fetchall()
print(rows[0][0])


cursor.execute('SELECT date, close FROM ilmn ORDER BY close DESC LIMIT 5;')
rows = cursor.fetchall()
for row in rows:
    print(row)

1
('2021-08-16 00:00:00', 510.54473876953125)
('2021-08-17 00:00:00', 508.6478576660156)
('2021-08-13 00:00:00', 503.2295837402344)
('2021-08-05 00:00:00', 501.6342468261719)
('2021-08-12 00:00:00', 496.70233154296875)


And now we've ingested the data into our database!